In [ ]:
import numpy as np
import torch


import matplotlib.pyplot as plt

In [ ]:
opt = torch.rand(1000,50)
ipt_batch = torch.rand(1000,50)

In [ ]:
torch.norm(opt,dim=0)

In [ ]:

def infonce(opt, ipt_batch, t=100):
    
    _, d = opt.shape
    Ln = 0
    for i in range(d):
        
        index = [x for x in range(d) if x!=i]
        a = [torch.exp(torch.norm(ipt_batch.T@opt[:, k:k+1])/t) for k in index]
        denominator = torch.stack(a).sum() + 1e-12

        numerator = torch.exp(torch.norm(ipt_batch.T@opt[:, i:i+1])/t)
        Ln += torch.log(numerator/denominator)
    return Ln.sum()/d

In [ ]:
def infonce1(opt, ipt_batch, t=100):
    
    _, d = opt.shape
    
    Ln = 0
    for i in range(d):
        index = [x for x in range(d) if x!=i]
        denominator = torch.exp(torch.norm(ipt_batch.T@opt[:, index], dim=0)/t).sum() + 1e-12
        
        numerator = torch.exp(torch.norm(ipt_batch.T@opt[:, i:i+1])/t)
        Ln += torch.log(numerator/denominator)
    return Ln.sum()/d

In [ ]:
def infonce2(opt, ipt_batch, t=100):
    
    _, d = opt.shape
    
    f = torch.exp(torch.norm(ipt_batch.T@opt, dim=0)/t)
    Ln = 0
    for i in range(d):
        index = [x for x in range(d) if x!=i]
        denominator = f[index].sum() + 1e-12
        
        numerator = f[i]
        Ln += torch.log(numerator/denominator)
    return Ln.sum()/d

In [ ]:
def infonce3(opt, ipt_batch, t=100):
    
    _, d = opt.shape
    
    f = torch.norm(ipt_batch.T@opt, dim=0)
    Ln = 0
    for i in range(d):
        index = [x for x in range(d) if x!=i]
        denominator = torch.exp(f[index]/t).sum() + 1e-12
        
        numerator = torch.exp(f[i]/t)
        Ln += torch.log(numerator/denominator)
    return Ln.sum()/d

In [ ]:
infonce(opt, ipt_batch)

In [ ]:
infonce3(opt,ipt_batch)

In [ ]:
infonce2(opt, ipt_batch)

In [ ]:
import torch

def least_squares_slope_ratio(x_tensor, y_tensor):
    # Convert input data to PyTorch tensors
#     x_tensor = torch.tensor(x, dtype=torch.float32)
#     y_tensor = torch.tensor(y, dtype=torch.float32)
    
    # Calculate the mean of x and y
    x_mean = torch.mean(x_tensor)
    y_mean = torch.mean(y_tensor)
    
    # Calculate the covariance and variance
    covariance = torch.mean((x_tensor - x_mean) * (y_tensor - y_mean))
    x_variance = torch.mean((x_tensor - x_mean) ** 2)
    
    # Estimate the slope 'a'
    slope = covariance / x_variance
    
    # Calculate the intercept 'b'
    intercept = y_mean - slope * x_mean
    
    return slope.item(), intercept.item()  # Convert results to Python floats



In [ ]:
# Example usage:
x_data = torch.rand(1000).to(torch.float32)
y_data = torch.rand(1000).to(torch.float32)

slope, intercept = least_squares_slope_ratio(x_data, y_data)
print("Estimated Slope (a):", slope)
print("Estimated Intercept (b):", intercept)


In [ ]:
def calculate_ols_slope(opt):
    
    # Calculate the OLS coefficients
    X_mean = torch.mean(opt[:,0])
    Y_mean = torch.mean(opt[:,1])
    
    num = torch.sum((X - X_mean) * (Y - Y_mean))
    den = torch.sum((X - X_mean) ** 2)
    
    slope = num / den
    
    return slope

In [ ]:
xy = torch.stack([x_data, y_data])

In [ ]:
calculate_ols_slope(xy)

In [ ]:
a = torch.corrcoef(torch.rand(10,100))
print(a)

In [ ]:
import torch

In [ ]:
opt = torch.randn(1000,10)

In [ ]:
def kurtosis(opt):
    # Calculate the mean and standard deviation of the data
    mean = torch.mean(opt, axis=0)
    std = torch.std(opt, axis=0)

    # Calculate the fourth central moment (raw kurtosis)
    fourth_moment = torch.mean(((opt - mean)/std)**4, axis=0)



    return torch.mean(fourth_moment)

In [ ]:
kurtosis(opt)

In [ ]:
import nibabel as nib
import numpy as np

In [ ]:
nifi = np.array(nib.load('../mri_data/fastica.nii').dataobj)

In [ ]:
a = nifi.reshape(53*63*52, 50)

In [ ]:
x = (a - a.mean(axis=0))/a.std(axis=0)

In [ ]:
nifiti = x.reshape(53,63,52,50)

In [ ]:
data_id = os.listdir('/data/qneuromark/Data/FBIRN/Data_BIDS/Raw_Data/')
path_list = []
for i in data_id:
    nii_path = '/data/qneuromark/Data/FBIRN/Data_BIDS/Raw_Data/' + i + '/ses_01/func/SM.nii'
    path_list.append(nii_path)
    
b = nib.load(path_list[2])
new_image = nib.Nifti1Image(nifiti, affine=b.affine, header=b.header)
nib.save(new_image, '/data/users2/yxiao11/model/ICA/mri_data/fastica.nii')

In [ ]:
import torch

In [ ]:
def my_loss(data):
    """
    Compute the kurtosis of a set of data using PyTorch.

    Args:
    data (torch.Tensor): A 1D tensor containing the data.

    Returns:
    float: The kurtosis of the data.
    """
    # Calculate the mean and standard deviation of the data
    mean = torch.mean(data, axis=0)
    std = torch.std(data, axis=0)

    # Calculate the fourth central moment (raw kurtosis)
    fourth_moment = torch.mean((data - mean)**4, axis=0)

    # Calculate the kurtosis using the fourth central moment
    n = data.size(0)
    kurtosis = fourth_moment / (std**4) - 3.0

    return torch.sum(-kurtosis**2)
#     return kurtosis

In [ ]:
def kurtosis(opt):
    # Calculate the mean and standard deviation of the data
    mean = torch.mean(opt, axis=0)
    std = torch.std(opt, axis=0)

    # Calculate the fourth central moment (raw kurtosis)
    kurtosis = torch.mean(((opt-mean)/std)**4, axis=0)-3
    return torch.mean(kurtosis)
#     # Calculate the kurtosis using the fourth central moment

#     kurtosis = fourth_moment / (std**4) - 3.0

#     return torch.mean(-kurtosis**2)
# #     return kurtosis

In [ ]:
x = torch.randn(10000,1)

In [ ]:
my_loss(x)

In [ ]:
kurtosis(x)

In [ ]:
import os
import nibabel as nib
import numpy as np


In [ ]:
mixture = get_mixture(path_list, 50)

In [ ]:
mixture.shape

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import matrix_rank, inv
from numpy.random import permutation
from scipy.linalg import eigh
from scipy.linalg import norm as mnorm
import ica

In [ ]:
data_id = os.listdir('/data/qneuromark/Data/FBIRN/Data_BIDS/Raw_Data/')
path_list = []
for i in data_id:
    nii_path = '/data/qneuromark/Data/FBIRN/Data_BIDS/Raw_Data/' + i + '/ses_01/func/SM.nii'
    path_list.append(nii_path)
    
def get_mixture(path_list, dims_remain):
    print('getting file:', 0)
    mri = nib.load(path_list[0])
    data = np.asanyarray(mri.dataobj)
    list_x = []
    for j in range(int(data.shape[-1])):
        list_x.append(np.expand_dims(data[:,:,:,j].flatten(),1))
    mixture = np.concatenate(list_x,axis=1).T
    
    for i in range(1, 5):
        print('getting file:', i)
        mri = nib.load(path_list[i])
        data = np.asanyarray(mri.dataobj)
        list_x = []
        for j in range(int(data.shape[-1])):
            list_x.append(np.expand_dims(data[:,:,:,j].flatten(),1))

        conc_x = np.concatenate(list_x,axis=1)
        mixture = np.concatenate((mixture, conc_x.T), axis=0)
        
    del mri, data, list_x, conc_x
#     mixture = np.float64(mixture.T)
    print('the shape of mixture is:', mixture.shape)
    print('whitening')
    mixture, white, dewhite = ica.pca_whiten(mixture, dims_remain)
    mixture = torch.from_numpy(mixture).type(torch.float32)
    print('done')
    return mixture, white, dewhite

In [ ]:
myfmri = np.asanyarray(nib.load('/data/users2/yxiao11/SM.nii').dataobj)

In [ ]:
myfmri.shape

In [ ]:
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(mixture[10,:].reshape(53,63,52)[:,:,26])

In [16]:
import nibabel as nib
import numpy as np
import os

In [29]:
mri = np.asanyarray(nib.load('../mri_data/infonce50.nii').dataobj)

In [30]:
mri.shape

(53, 63, 52, 50)

In [31]:
mri = mri.reshape(53*63*52, 50).T

In [32]:
from sklearn import preprocessing

In [33]:
mri = preprocessing.StandardScaler().fit_transform(mri.T)
mri = (np.diag(1/np.abs(mri.T).max(axis=1))@mri.T).astype('float32')

In [34]:
mri.shape

(50, 173628)

In [35]:
# data_id = os.listdir('/data/qneuromark/Data/FBIRN/Data_BIDS/Raw_Data/')
# path_list = []
# for i in data_id:
#     nii_path = '/data/qneuromark/Data/FBIRN/Data_BIDS/Raw_Data/' + i + '/ses_01/func/SM.nii'
#     path_list.append(nii_path)

b = nib.load('/data/users2/yxiao11/model/ICA/mri_data/template.nii.gz')

In [36]:
a = mri.T
a = a.reshape(53,63,52,50)

In [37]:
nifiti = np.array(a)
new_image = nib.Nifti1Image(nifiti, affine=b.affine, header=b.header)
# nib.save(new_image, 'mri_data/adam_meanNN_' + str(dims_remain) +'_' + str(batch_size) + 'batch')
nib.save(new_image, '/data/users2/yxiao11/model/ICA/mri_data/infonce_50.nii')

In [38]:
a.max()

1.0

In [39]:
a.min()

-1.0